Using kernel `conda_pytorch_latest_p36`

In [1]:
# !pip install transformers
# !pip install datasets

In [2]:
import sys
sys.path.append('../../../')

In [8]:
from pathlib import Path
import os
import random
import pickle

In [4]:
import pandas as pd
import numpy as np
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import torch

In [5]:
from deep.constants import *

In [23]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [29]:
a = pd.read_pickle('/Users/stefano/Downloads/test_predictions.pickle')
b = sigmoid(a)

In [30]:
a.shape

torch.Size([11617, 10])

In [31]:
a

tensor([[ 0.1980,  0.0710,  0.2459,  ..., -0.2403,  0.0762,  0.0413],
        [ 0.3667,  0.1059,  0.3353,  ..., -0.0385,  0.0689, -0.2443],
        [ 0.4063, -0.0352,  0.2770,  ..., -0.1974,  0.0131, -0.0497],
        ...,
        [ 0.2346,  0.2211,  0.4581,  ..., -0.0631,  0.0730, -0.0727],
        [ 0.1940,  0.1128,  0.3020,  ..., -0.0594,  0.1630,  0.0610],
        [ 0.0056,  0.1211,  0.3832,  ..., -0.1178,  0.1126, -0.1457]])

In [27]:
a

tensor([[0.5493, 0.5177, 0.5612,  ..., 0.4402, 0.5190, 0.5103],
        [0.5907, 0.5265, 0.5830,  ..., 0.4904, 0.5172, 0.4392],
        [0.6002, 0.4912, 0.5688,  ..., 0.4508, 0.5033, 0.4876],
        ...,
        [0.5584, 0.5551, 0.6126,  ..., 0.4842, 0.5182, 0.4818],
        [0.5483, 0.5282, 0.5749,  ..., 0.4852, 0.5407, 0.5152],
        [0.5014, 0.5302, 0.5947,  ..., 0.4706, 0.5281, 0.4636]])